## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
use the beautifulsoup for scraping data from website: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M



### Import Libraries

In [20]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Scrap List of postal codes of Canada wiki page content by using requests library and parese with BeautifulSoup

In [21]:
# download data from website and create instance of BeautifulSoup class for parsing the data. 
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
webcontent = requests.get(URL).text
Canada_data = BeautifulSoup(webcontent, 'lxml')

### Create dataframe from the postal code data from the HTML table on the site

In [29]:
# create a new set of cloumns and then a Dataframe 
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

# Based on the inspect on browser data is available at the tags - div and class= 'mw-parser-output'
#loop through the data to find postcode, borough, neighborhood sections
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

#strip '\n' from neighborhood columns and add (combine rather) neighborhood column values for the same Borough types
for tr in table.find_all('tr'):
    col = 0
    for td in tr.find_all('td'):
        if col == 0:
            postcode = td.text
            col = col + 1
        elif col == 1:
            borough = td.text
            col = col + 1
        elif col == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
            break
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean up dataframe - remove 'Not assigned' Borough values and then reset the index
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)

#replace 
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print(df.shape)
df.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Data Cleaning
Drop "None" rows in DataFrame

Drop any row which contains 'Not assigned' value

All "Not assigned" will be replace to 'NaN' using numpy for convenience.

In [24]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [28]:
print(df.shape)
df.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
